In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as ps
import sklearn.metrics as skm
import sklearn.model_selection as sms
import sklearn.tree as st
import sklearn.neighbors as snn
import sklearn.preprocessing as sp
import sklearn.ensemble as se
import numpy as np
import pandas.io.sql as sqlio

In [26]:
conn = ps.connect( # Connect to Redshift Serverless Database
    host='galaxy2-dw.169592149406.us-east-1.redshift-serverless.amazonaws.com',
    port=5439,
    database='dev',
    user='evanf',
    password='yayAtHa3'
)

In [27]:
query = 'SELECT deaths, star_bits, coins, difficulty, lives_found, avg_hits, bossIndex, prankIndex, enemy_rank, trap_rank, power_rank FROM diff_model'
data = sqlio.read_sql_query(query, conn)

In [28]:
print(data.head(5))

   deaths  star_bits  coins   difficulty  lives_found  avg_hits  bossindex  \
0       0         79     21  Easy-Medium            1       0.0        1.0   
1       0        161     29  Easy-Medium            0       0.0        0.0   
2       0          8      0       Expert            1       0.0        0.0   
3       0         99      0  Medium-Hard            0       0.0        0.0   
4       0         91      4  Medium-Hard            0       0.0        0.0   

   prankindex  enemy_rank  trap_rank  power_rank  
0         0.0        52.0       11.0        20.0  
1         0.0        50.0       23.0        12.0  
2         1.0       100.0       44.0        20.0  
3         1.0        55.0       50.0         4.0  
4         1.0        18.0       50.0        20.0  


In [66]:
x = data[['deaths', 'avg_hits', 'bossindex', 'prankindex', 'enemy_rank', 'trap_rank', 'power_rank']]
y = data['difficulty']

x_train, x_test, y_train, y_test = sms.train_test_split(x, y, test_size = 0.3, random_state = 102)

diff_dt = st.DecisionTreeClassifier(max_depth = 6) # Use a Decision Tree Classifier this time
diff_dt.fit(x_train, y_train)
difficulty_dt_pred = diff_dt.predict(x_test)

print(skm.classification_report(y_test, difficulty_dt_pred)) # Find the results of the model
print(skm.confusion_matrix(y_test, difficulty_dt_pred)) # Print the Confusion matrix of the model

difficulty_rf = se.RandomForestClassifier(n_estimators = 300) # Use 300 trees
difficulty_rf.fit(x_train, y_train)
difficulty_rf_pred = difficulty_rf.predict(x_test)

#print(skm.classification_report(y_test, difficulty_rf_pred)) # Find the results of the model
#print(skm.confusion_matrix(y_test, difficulty_rf_pred)) # Print the Confusion matrix of the model

8
              precision    recall  f1-score   support

        Easy       0.50      0.25      0.33         4
 Easy-Medium       0.25      0.33      0.29         3
      Expert       0.00      0.00      0.00         1
        Hard       0.33      0.25      0.29         8
      Harder       0.00      0.00      0.00         2
      Medium       0.33      0.57      0.42         7
 Medium-Hard       0.56      0.42      0.48        12

    accuracy                           0.35        37
   macro avg       0.28      0.26      0.26        37
weighted avg       0.39      0.35      0.36        37

[[1 0 0 0 0 2 1]
 [0 1 0 1 0 1 0]
 [0 0 0 0 0 1 0]
 [1 0 0 2 1 2 2]
 [0 0 0 0 0 1 1]
 [0 2 0 1 0 4 0]
 [0 1 0 2 3 1 5]]


C:\Users\evanf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\evanf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\evanf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1469: Unde

Not very good results besides for medium-hard levels but when lives_found and major_sections are taken out the results are a little better and then even better when taking out coins and star bits
Also see there are not a lot of data points for every difficulty to test so precision is 0 and accuracy is very low
Maybe need to incorporate Mario Galaxy 1 data points we can have more data to build a better model

In [50]:
# Try using KNN and see if results are improved --> they are vastly improved but still not great, need more data points

# First we need to scale the data to be able to calculate distance between points
scaler = sp.StandardScaler()
scaler.fit(x)
scaled_diff = scaler.transform(x)
X_scaled = pd.DataFrame(scaled_diff, columns = x.columns)

# Now train_test_split the scaled data then fit the model
X_trains, X_tests, y_trains, y_tests = sms.train_test_split(X_scaled, y, test_size = 0.3, random_state = 102)
diff_knn = snn.KNeighborsClassifier(n_neighbors = 8) # Use a value of 8 for K for best results

diff_knn.fit(X_trains, y_trains)
knn_pred = diff_knn.predict(X_tests)

print(skm.classification_report(y_tests, knn_pred)) # Print the report for KNN
print(skm.confusion_matrix(y_tests, knn_pred)) # Print the Confusion matrix for KNN and compare it to tree methods

print(skm.accuracy_score(y_test, knn_pred)) # 0/very low precision and accuracy for harder and expert difficulties because of lack of data points

              precision    recall  f1-score   support

        Easy       0.25      0.50      0.33         4
 Easy-Medium       0.50      0.33      0.40         3
      Expert       0.25      1.00      0.40         1
        Hard       0.50      0.12      0.20         8
      Harder       0.00      0.00      0.00         2
      Medium       0.44      0.57      0.50         7
 Medium-Hard       0.60      0.50      0.55        12

    accuracy                           0.41        37
   macro avg       0.36      0.43      0.34        37
weighted avg       0.46      0.41      0.39        37

[[2 0 0 0 0 1 1]
 [1 1 1 0 0 0 0]
 [0 0 1 0 0 0 0]
 [2 0 2 1 1 0 2]
 [0 0 0 0 0 2 0]
 [1 1 0 0 0 4 1]
 [2 0 0 1 1 2 6]]
0.40540540540540543


In [58]:
# Use KNN but this time keep out Expert, Harder --> results are worse
x = data[data.difficulty != 'Harder']
x = x[x.difficulty != 'Expert']
x_mod = x[['deaths', 'avg_hits', 'bossindex', 'prankindex', 'enemy_rank', 'trap_rank', 'power_rank']]
y = x['difficulty']

# First we need to scale the data to be able to calculate distance between points
scaler = sp.StandardScaler()
scaler.fit(x_mod)
scaled_diff = scaler.transform(x_mod)
X_scaled = pd.DataFrame(scaled_diff, columns = x_mod.columns)

# Now train_test_split the scaled data then fit the model
X_trains, X_tests, y_trains, y_tests = sms.train_test_split(X_scaled, y, test_size = 0.3, random_state = 102)
diff_knn = snn.KNeighborsClassifier(n_neighbors = 8) # Use a value of 8 for K for best results

diff_knn.fit(X_trains, y_trains)
knn_pred = diff_knn.predict(X_tests)

print(skm.classification_report(y_tests, knn_pred)) # Print the report for KNN
print(skm.confusion_matrix(y_tests, knn_pred)) # Print the Confusion matrix for KNN and compare it to tree methods

              precision    recall  f1-score   support

        Easy       0.00      0.00      0.00         1
 Easy-Medium       0.00      0.00      0.00         2
        Hard       0.50      0.14      0.22         7
      Medium       0.33      0.15      0.21        13
 Medium-Hard       0.33      0.56      0.42         9

    accuracy                           0.25        32
   macro avg       0.23      0.17      0.17        32
weighted avg       0.34      0.25      0.25        32

[[0 1 0 0 0]
 [0 0 0 1 1]
 [1 2 1 1 2]
 [2 1 1 2 7]
 [1 1 0 2 5]]
